## Context:
The dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. Features consist of hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V) to predict the net hourly electrical energy output (EP) of the plant.
A combined cycle power plant (CCPP) is composed of gas turbines (GT), steam turbines (ST) and heat recovery steam generators. In a CCPP, the electricity is generated by gas and steam turbines, which are combined in one cycle, and is transferred from one turbine to another. While the Vacuum is colected from and has effect on the Steam Turbine, he other three of the ambient variables effect the GT performance.
For comparability with our baseline studies, and to allow 5x2 fold statistical tests be carried out, we provide the data shuffled five times. For each shuffling 2-fold CV is carried out and the resulting 10 measurements are used for statistical testing.

## Attribute Information:

Features consist of hourly average ambient variables
- Temperature (T) in the range 1.81°C and 37.11°C,
- Ambient Pressure (AP) in the range 992.89-1033.30 milibar,
- Relative Humidity (RH) in the range 25.56% to 100.16%
- Exhaust Vacuum (V) in teh range 25.36-81.56 cm Hg
- Net hourly electrical energy output (EP) 420.26-495.76 MW
The averages are taken from various sensors located around the plant that record the ambient variables every second. The variables are given without normalization.

### Source - https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant

In [2]:
from pyforest import *

In [1]:
import tensorflow as tf

In [4]:
tf.__version__

<IPython.core.display.Javascript object>

'1.14.0'

In [6]:
df=pd.read_excel('PowerPlant.xlsx')
df.head()

<IPython.core.display.Javascript object>

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
AT    9568 non-null float64
V     9568 non-null float64
AP    9568 non-null float64
RH    9568 non-null float64
PE    9568 non-null float64
dtypes: float64(5)
memory usage: 373.9 KB


In [8]:
df.shape

(9568, 5)

In [9]:
df.isnull().sum()

AT    0
V     0
AP    0
RH    0
PE    0
dtype: int64

In [13]:
df.corr()['PE'].sort_values()

AT   -0.948128
V    -0.869780
RH    0.389794
AP    0.518429
PE    1.000000
Name: PE, dtype: float64

In [28]:
X=df.drop('PE',1).values
y=df.PE.values

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=.3, random_state=0)
X_train.shape

(6697, 4)

In [30]:
from keras.layers import Dense
from keras.layers import ReLU
from keras.activations import sigmoid
from keras.models import Sequential

In [43]:
ann=Sequential()

In [44]:
ann.add(Dense(units=6, activation='relu', kernel_initializer='he_uniform'))

In [45]:
ann.add(Dense(units=6, activation='relu', kernel_initializer='glorot_uniform'))

In [46]:
ann.add(Dense(units=1))

In [47]:
ann.compile(optimizer='adam', loss='mean_squared_error')

In [48]:
ann.fit(X_train,y_train, batch_size=32, epochs=100)

Epoch 1/100
6697/6697 [==============================] - 1s 126us/step - loss: 453678.3657
Epoch 2/100
6697/6697 [==============================] - 1s 99us/step - loss: 1770.4373
Epoch 3/100
6697/6697 [==============================] - 1s 136us/step - loss: 349.7719
Epoch 4/100
6697/6697 [==============================] - 1s 106us/step - loss: 345.4777
Epoch 5/100
6697/6697 [==============================] - 1s 93us/step - loss: 340.3419
Epoch 6/100
6697/6697 [==============================] - 1s 99us/step - loss: 334.4429
Epoch 7/100
6697/6697 [==============================] - 1s 92us/step - loss: 328.0096
Epoch 8/100
6697/6697 [==============================] - 1s 85us/step - loss: 320.4216
Epoch 9/100
6697/6697 [==============================] - 1s 77us/step - loss: 312.6931
Epoch 10/100
6697/6697 [==============================] - 1s 93us/step - loss: 303.7964
Epoch 11/100
6697/6697 [==============================] - 0s 72us/step - loss: 294.0631
Epoch 12/100
6697/6697 [==========

6697/6697 [==============================] - 1s 95us/step - loss: 27.4087
Epoch 95/100
6697/6697 [==============================] - 1s 107us/step - loss: 27.3828
Epoch 96/100
6697/6697 [==============================] - 1s 88us/step - loss: 27.5914
Epoch 97/100
6697/6697 [==============================] - ETA: 0s - loss: 27.0654 ETA:  - 0s 75us/step - loss: 27.3965
Epoch 98/100
6697/6697 [==============================] - 0s 74us/step - loss: 27.0972
Epoch 99/100
6697/6697 [==============================] - 1s 76us/step - loss: 28.4375
Epoch 100/100
6697/6697 [==============================] - 1s 97us/step - loss: 29.6669


In [55]:
ypred=ann.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((ypred.reshape(-1,1), y_test.reshape(-1,1)),1))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[[429.5  431.23]
 [460.8  460.01]
 [464.4  461.14]
 ...
 [432.69 426.86]
 [433.83 433.4 ]
 [437.21 441.94]]


In [62]:
from sklearn.metrics import mean_squared_error, r2_score
print('r2 test:',r2_score(y_test,ypred))
print('r2 train:',r2_score(y_train,ann.predict(X_train)))
print('rmse:', np.sqrt(mean_squared_error(y_test,ypred)))

r2 test: 0.9039725434777127
r2 train: 0.9030010467818839


<IPython.core.display.Javascript object>

rmse: 5.301592770052205
